# Analysis of Vital Signs

In [1]:
library(tidyverse)
library(fasttime)
library(JM)

Warning message in as.POSIXlt.POSIXct(Sys.time()):
“unknown timezone 'zone/tz/2020d.1.0/zoneinfo/America/New_York'”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: MASS


Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:dplyr’:

    collapse


Loading required package: splines

Loading required package: survival



In [2]:
base_dir <- '~/Google Drive/Projects/Haimovich_Pressors_Tmp'

In [3]:
vitals_df <- read_csv(file.path(base_dir, 'vitals.csv.gz'))

Parsed with column specification:
cols(
  subject_id = col_double(),
  hadm_id = col_double(),
  stay_id = col_double(),
  charttime = col_datetime(format = ""),
  valuenum = col_double(),
  valueuom = col_character(),
  itemid = col_double(),
  assignment = col_character()
)



In [4]:
output_df <- read.csv(file.path(base_dir, 'output.csv'))

In [5]:
# 20  (1am), 3 hrs
vitals_merged_df <- vitals_df %>%
    filter(stay_id %in% output_df$stay_id) %>%
    left_join(dplyr::select(output_df, stay_id, course_id, pressor, intime, starttime, endtime, MORETHAN24HOURS, RESPONSE, STATUS), by = "stay_id") %>%
    mutate(charttime = fastPOSIXct(charttime), 
           starttime = fastPOSIXct(starttime), 
           intime = fastPOSIXct(intime)
          ) %>%
    mutate(
          start_pressortime = difftime(charttime, 
                       starttime, units="hours"),
          start_intime = difftime(charttime, 
                       intime, units="hours")) 

In [6]:
dim(vitals_merged_df)

[1] 4429027      18

In [17]:
assignments <- c('HR', 'MBP', 'RR', 'SPO2', 'TEMPC')

In [18]:
result_assignment_list <- list()

In [19]:
for(assignment_id in assignments){
    vitals_subset_df <- vitals_merged_df %>% 
    mutate(start_intime = as.numeric(start_intime)) %>%
    filter(assignment == assignment_id)  %>% 
    filter(charttime < starttime) %>%
    head(14000)

    survival_df <- output_df %>%
        filter(course_id %in% unique(vitals_subset_df$course_id)) %>%
        mutate(intime = fastPOSIXct(intime)) %>%
        mutate(start_intime = difftime(starttime, 
                           intime, units="hours")) %>% 
        mutate(start_intime = as.numeric(start_intime)) %>%
        mutate(dummy = 0)

    # bug in JM, so arrange is needed
    ctrl <- lmeControl(opt='optim');
    lmeFit.p1 <- lme(valuenum ~ start_intime, random = ~ start_intime | course_id,
                     data = vitals_subset_df %>% 
                     arrange(course_id, start_intime),control=ctrl)

    survFit.p1 <- coxph(Surv(start_intime, RESPONSE) ~ 1, x = TRUE,
                        data = survival_df %>% arrange(course_id, start_intime)
                        )  

    jointFit.p1 <- jointModel(lmeFit.p1, survFit.p1, timeVar = "start_intime",
        method = "Cox-PH-GH")

    result_assignment_list[[assignment_id]] <- confint(jointFit.p1)
    
}

Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructInt(lmeSt, lmePars):
“Singular precision matrix in level -1, block 196”
Warning message in logLik.lmeStructIn

ERROR: Error in MEestimate(lmeSt, grps): Singularity in backsolve at level 0, block 1


In [21]:
confint(jointFit.p1)

,2.5 %,est.,97.5 %
Y.(Intercept),97.5478978412,97.6535098577,97.7591218742
Y.start_intime,-0.0008356584,-0.0001903008,0.0004550568
T.Assoct,-0.0224348590,-0.0214346027,-0.0204343464


In [31]:
result_assignment_df <- bind_rows(result_assignment_list, .id='ASSIGNMENT')

In [32]:
#4000 - 82
#8000 - 112
#16000 - 1582

#60000

In [33]:
#result_assignment_df %>% write.csv("~/Dropbox/out.csv")

In [35]:
result_assignment_list[['HR']]

,2.5 %,est.,97.5 %
Y.(Intercept),9.212191e+01,92.568555943,93.015204876
Y.start_intime,5.770953e-04,0.003565061,0.006553028
T.Assoct,5.083427e-03,0.006100946,0.007118465


In [1]:
read.csv("~/Dropbox/out.csv")

X,ASSIGNMENT,HR,MBP,RR,SPO2
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,9.212191e+01,77.797542468,19.387493874,97.5478978412
2,1,5.770953e-04,0.002598712,0.006744429,-0.0008356584
3,1,5.083427e-03,-0.029030658,0.014873482,-0.0224348590
4,1,9.256856e+01,77.898353828,19.586168455,97.6535098577
5,1,3.565061e-03,0.005572834,0.007855484,-0.0001903008
6,1,6.100946e-03,-0.027749720,0.019550394,-0.0214346027
7,1,9.301520e+01,77.999165188,19.784843036,97.7591218742
8,1,6.553028e-03,0.008546957,0.008966539,0.0004550568
9,1,7.118465e-03,-0.026468783,0.024227306,-0.0204343464
